In [1]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera

from scipy.signal import find_peaks, butter, filtfilt

import time
import matplotlib.pyplot as plt
import skimage as ski
import numpy as np

# Camera resolution

res = (640, 480)

CAMERA_CENTER = res[0] // 2
       
from pwm import PWM

## Setup servo and motor PWM signal to control the steering angle and the speed.

The values specified period and duty cycle ar in nanoseconds.

Servo and motor controlled follow the following protocol:
  1. Control signal is read 50 times per second by each device (servo and speed controller). Thus period is 20 ms => 20000000 ns
  2. To send the minimum value (full left for servo and break for motor) the duty cycle should be 1 ms => 1000000 ns
  3. Maximum value corresponds to a duty cycle of 2 ms => 2000000 ns
  
However, these devices are not perfect and you can set a duty cycle outside this range and still get a response. I would recoment to play with it and find are the limits in your case. Pay attention that servo is limited by the mechanical construction how much it can turn the wheels. Do not set these limits to a value which servo cannot execute as it will try to do that and eventually burn.

In [14]:
# Enable servo
SERVO_MIDDLE = 1500000

servo = PWM(1)
servo.period = 20000000
servo.duty_cycle = SERVO_MIDDLE
servo.enable = True

In [15]:
# Enable servo
MOTOR_BRAKE = 1000000

motor = PWM(0)
motor.period = 20000000
motor.duty_cycle = MOTOR_BRAKE
motor.enable = True

In [16]:
motor.duty_cycle = MOTOR_BRAKE 

## The self driving code

Quick explanation on how this algorithm works:
  1. Initialize the camera, use a frame rate of 10 to 20 fps
  2. Get the coefficients of the butterworth filter of order 3 and cut-off frequency of 0.1 (change cut of frequency if the peak functions is not working)
  3. For each frame captured from the camera:
    1. Get the blak and white part of the image
    2. Select line 195 to use for lane detection (this number depends on your camera angle)
    3. Filter the line and find the peaks that are greater than 160. White parts of the image have higher values than the black. The threshold of 160 depends on the ambient light, lower if the room is darker
    4. Find the peaks left and right of the camera center, these peaks should corresponds to the white lines delimiting the track,
    5. Compute the center point between these lines. This is where we want our car
    6. Compute the error by substracting the center point from camera middle and multiply by 5000. 5000 is our P gain.
    7. Adjust the duty cycle of the servo such that the

In [17]:
# Run a track detection algorithm on a single horizontal line.
# Uses YUV420 image format as the Y component corresponds to image intensity (gray image)
# and thus there is no need to convert from RGB to BW

RUN_TIMER = 17 # seconds
history=[]

camera = PiCamera()
        
# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 10

# Initialize the buffer and start capturing
rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

# Measure the time needed to process 300 images to estimate the FPS
t = time.time()

# To filter the noise in the image we use a 3rd order Butterworth filter

# Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
b, a = butter(3, 0.1)

line_pos    = CAMERA_CENTER
first_frame = True

# start car
motor.duty_cycle = MOTOR_BRAKE + 120000
track_width = 0
MOTOR_MAX = 310000
MOTOR_STRAIGHT = MOTOR_MAX
SPEED_COMFY = 240000
MOTOR_TURN_INIT = 150000
MOTOR_TURN = MOTOR_TURN_INIT
TURN_COMFY = 250000
turn_ratio_start = 1.9
turn_ratio_end = 1.3
inc = 0.1
turn_ratio = turn_ratio_start
for f in stream:
    if first_frame:
        first_frame = False
        # Reset the buffer for the next image
        rawCapture.truncate(0)
        continue
    
    # Stop after RUN_TIMER seconds
    if (time.time() - t) > RUN_TIMER:
        break
        
    # Get the intensity component of the image (a trick to get black and white images)
    I = f.array[:, :, 0]
    
    # Reset the buffer for the next image
    rawCapture.truncate(0)
    
    # Select a horizontal line in the middle of the image
    L = I[195, :]

    # Smooth the transitions so we can detect the peaks 
    Lf = filtfilt(b, a, L)
    history.append(Lf)
    
    # Find peaks which are higher than 0.5
    p = find_peaks(Lf, height=160)
      
    peaks = p[0]
    
    
    line_left   = None
    line_right  = None
    peaks_left  = peaks[peaks < CAMERA_CENTER]
    peaks_right = peaks[peaks > CAMERA_CENTER]
    
    # state machine
    
    if peaks_left.size:
        line_left = peaks_left.max()

    if peaks_right.size:
        line_right = peaks_right.min()
        
    if line_left and line_right:
        line_pos    = (line_left + line_right ) // 2
        track_width = line_right - line_left
        motor.duty_cycle = MOTOR_BRAKE + MOTOR_STRAIGHT
        if(MOTOR_STRAIGHT >= SPEED_COMFY):
            MOTOR_STRAIGHT -= 10000
        MOTOR_TURN = MOTOR_TURN_INIT
        turn_ratio = turn_ratio_start
    elif line_left and not line_right:
#         track_width = 400
        line_pos    = line_left + int(track_width/turn_ratio)
#         line_pos    = line_left + int(track_width / 2)
        motor.duty_cycle = MOTOR_BRAKE + MOTOR_TURN
        MOTOR_STRAIGHT = MOTOR_MAX
        if(MOTOR_TURN <= TURN_COMFY):
            MOTOR_TURN += 10000
        if(turn_ratio >= turn_ratio_end):
            turn_ratio -= inc
    elif not line_left and line_right:
#         track_width = 400
        line_pos    = line_right - int(track_width/turn_ratio)
#         line_pos    = line_right - int(track_width / 2)
        motor.duty_cycle = MOTOR_BRAKE + MOTOR_TURN
        MOTOR_STRAIGHT = MOTOR_MAX
        if(MOTOR_TURN <= TURN_COMFY):
            MOTOR_TURN += 10000
        if(turn_ratio >= turn_ratio_end):
            turn_ratio -= inc
    else:
        track_width = 400
        print("no line")
        turn_ratio = turn_ratio_start
        
    print(line_pos, peaks)

        
    DUTY_CYCLE = SERVO_MIDDLE + 5000 * (CAMERA_CENTER - line_pos)
    if DUTY_CYCLE > 2000000:
        DUTY_CYCLE = 2000000
    if DUTY_CYCLE < 1000000:
        DUTY_CYCLE = 1000000
        
    servo.duty_cycle =  DUTY_CYCLE
        
#Initialize lines position
#Check which lines are closer them in the next frame

        
        #print(line_pos)
        
motor.duty_cycle = MOTOR_BRAKE    
    

# Release resources
stream.close()
rawCapture.close()
camera.close()

313 [ 41 585 615]
313 [ 40 586 615]
313 [ 41 585 614]
313 [ 43 583 613]
310 [ 39 581 610]
309 [ 39 579 609]
310 [ 41 580 611]
310 [ 40 580 610]
309 [ 37 582 609]
314 [ 35 594]
331 [37]
351 [13 41]
368 [ 8 40]
394 [45]
397 [48]
398 [49]
400 [51]
400 [51]
398 [49]
399 [50]
394 [45]
401 [52]
400 [51]
406 [57]
413 [64]
417 [68]
436 [87]
450 [101]
471 [112 122]
498 [132 149]
527 [155 178]
559 [183 210]
598 [214 231 249]
636 [248 267 287]
660 [271 291 311]
319 [299 320 340]
303 [324 340 356]
303 [325 351]
325 [314 336]
321 [332 353]
356 [368 387]
379 [391 409]
398 [411 427]
394 [407 424]
369 [382 402]
328 [341 360]
327 [298 314]
294 [281]
276 [246 263]
263 [233 250]
268 [238 255]
285 [253 272]
317 [284 304]
327 [318 337]
336 [346 364]
355 [365 382]
356 [367 384]
342 [353 371]
314 [325 344]
323 [293 312]
237 [226]
no line
237 []
no line
237 []
no line
237 []
428 [638]
410 [632]
390 [625]
364 [614]
351 [601]
329 [579]
282 [532 549]
102 [352 374 396 412 434 456 637]
310 [560 597]
463 [ 41  63  

In [18]:
# Release resources
stream.close()
rawCapture.close()
camera.close()
motor.duty_cycle = MOTOR_BRAKE   